In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
path = "drive/MyDrive/hangman/"

In [2]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import matplotlib.pyplot as plt 
# import math
# import random
# import string

# torch.manual_seed(1)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# f = open(path+"words_250000_train.txt", "r")

# words = []
# words

# for _ in range(227300):
#     word = f.readline()
#     word = word.split('\n')[0]
#     words.append(word)


# random.shuffle(words)
# train_words = words[0:200000]
# test_words = words[200000:227300]

# file = open(path+"words.txt","w")

# for word in words:
#     file.write(word+"\n")

# file.close()

# ALL_LETTERS = string.ascii_lowercase + '_'

# def letter_to_index(letter):
#     return ALL_LETTERS.find(letter)

# def char_to_vec(letter):
#     vec = torch.zeros(1, 27)
#     vec[0][letter_to_index(letter)] = 1
#     return vec.to(device)

# def word_to_tensor(word):
#     tensor = torch.zeros(len(word), 1, 27)
#     for i, letter in enumerate(word):
#         tensor[i][0][letter_to_index(letter)] = 1
#     return tensor.to(device)

# def get_freq_tensor(word, obscure_word, guessed):
#     freq_tensor = torch.zeros(1,26)
#     for l in word:
#         if l not in obscure_word:
#             freq_tensor[0][letter_to_index(l)] += 1
#     for i in range(26):
#         if guessed[0][i] == 1:
#             freq_tensor[0][i] = 0
#     freq_tensor /= freq_tensor.sum()
#     return freq_tensor.to(device)

# def guess_from_output(output, guessed):
#     for i in range(26):
#         guess_idx = output.kthvalue(26-i).indices[0].item()
#         if guessed[0][guess_idx] == 0:
#             break
#     return ALL_LETTERS[guess_idx]

# def guess_idx_from_output(output, guessed):
#     for i in range(26):
#         guess_idx = output.kthvalue(26-i).indices[0].item()
#         if guessed[0][guess_idx] == 0:
#             break
#     return guess_idx

# class LSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(LSTM, self).__init__()
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size

#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers).to(device)
#         self.fc = nn.Linear(hidden_size+26, num_classes).to(device)
        
#     def forward(self, x, guessed, h0=None, c0=None):
#         if h0 == None:
#             h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device) 
#             c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device) 
#         out, (hn,cn) = self.lstm(x, (h0,c0))
#         out = out[-1, :, :].to(device)
#         out = torch.cat((out,guessed), 1).to(device)
#         out = self.fc(out).to(device)
#         return out, (hn,cn)


# learning_rate = 0.002
# hidden_size = 512
# num_layers = 2

# logsoftmax = nn.LogSoftmax(dim=0)
# lstm = LSTM(27,hidden_size,num_layers,26)
# optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
# def criterion(pred, tar):
#     return -torch.dot(tar[0],logsoftmax(pred[0]))

# # lstm.load_state_dict(torch.load('abc'))


# def upd(loss):
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

# steps = 100
# def train(batch_size, epochs):
#     all_losses = []
#     current_loss = 0
#     attempts = 0
#     wrong_attempts = 0
#     won = 0
#     div = 0
#     for e in range(epochs):
#         random.shuffle(train_words)
#         for i in range(200000):
#             word = train_words[i]
#             guessed = torch.zeros(1,26).to(device)
#             obscure_word = '_'*len(word)
#             tar = get_freq_tensor(word,obscure_word,guessed)

#             if i>0 and i%steps == 0:
#                 # avg_loss = current_loss/steps
#                 # upd(avg_loss)
#                 print(f"epoch: {e}, {i/2000}% completed, avg_loss: {avg_loss}, avg_attempts: {attempts/steps}, avg_wrong_attempts: {wrong_attempts/steps}, won%: {(won*100)/steps}")
#                 won = 0
#                 attempts = 0
#                 wrong_attempts = 0
#                 all_losses.append(current_loss)
#                 # current_loss = 0

#             if i>0 and i%batch_size == 0:
#                 avg_loss = current_loss/div
#                 upd(avg_loss)
#                 div = 0
#                 # print(f"epoch: {e}, {i/2000}% completed, avg_loss: {avg_loss}, avg_attempts: {attempts/steps}, avg_wrong_attempts: {wrong_attempts/steps}")
#                 # attempts = 0
#                 # wrong_attempts = 0
#                 # all_losses.append(current_loss)
#                 current_loss = 0

#             x = len(word)
#             y = 0
#             h0 = 0
#             c0 = 0
#             cur_a = 0
#             while obscure_word.find('_') != -1:
#                 div+=1
#                 input = word_to_tensor(obscure_word).to(device)
#                 attempts += 1
#                 if type(h0) == int:
#                     output, _ = lstm(input, guessed)
#                 else:
#                     output, _ = lstm(input, guessed, h0, c0)

#                 loss = criterion(output, tar)
#                 current_loss += loss 
                
#                 guess = guess_from_output(output, guessed)
#                 guess_idx = guess_idx_from_output(output, guessed)
                
#                 # upd(loss)
#                 guessed[0][ALL_LETTERS.find(guess)] = 1
#                 if guess in word:
#                     x-=1
#                     y+=1
#                     for j in range(len(word)):
#                         if(word[j]==guess):
#                             obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
#                     tar = get_freq_tensor(word,obscure_word,guessed)
#                 else:
#                     wrong_attempts += 1
#                     cur_a+=1
#                 if (i+1)%1000 == 0:
#                     print(f"obscure_word: {obscure_word}, guess: {guess}")
#             if cur_a < 6:
#                 won+=1
#         torch.save(lstm.state_dict(), path+"result_dict_lstm_"+str(e))
#         torch.save(lstm, path+"result_model_lstm_"+str(e))
#     plt.figure()
#     plt.plot(all_losses)
#     plt.show()


# train(16, 10)




In [3]:
last_epoch = 1
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
import math
import random
import string

torch.manual_seed(1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


f = open(path+"words.txt", "r")

train_words = []
train_words


for _ in range(200000):
    word = f.readline()
    word = word.split('\n')[0]
    train_words.append(word)


ALL_LETTERS = string.ascii_lowercase + '_'

def letter_to_index(letter):
    return ALL_LETTERS.find(letter)

def char_to_vec(letter):
    vec = torch.zeros(1, 27)
    vec[0][letter_to_index(letter)] = 1
    return vec.to(device)

def word_to_tensor(word):
    tensor = torch.zeros(len(word), 1, 27)
    for i, letter in enumerate(word):
        tensor[i][0][letter_to_index(letter)] = 1
    return tensor.to(device)

def get_freq_tensor(word, obscure_word, guessed):
    freq_tensor = torch.zeros(1,26)
    for l in word:
        if l not in obscure_word:
            freq_tensor[0][letter_to_index(l)] += 1
    for i in range(26):
        if guessed[0][i] == 1:
            freq_tensor[0][i] = 0
    freq_tensor /= freq_tensor.sum()
    return freq_tensor.to(device)

def guess_from_output(output, guessed):
    for i in range(26):
        guess_idx = output.kthvalue(26-i).indices[0].item()
        if guessed[0][guess_idx] == 0:
            break
    return ALL_LETTERS[guess_idx]

def guess_idx_from_output(output, guessed):
    for i in range(26):
        guess_idx = output.kthvalue(26-i).indices[0].item()
        if guessed[0][guess_idx] == 0:
            break
    return guess_idx

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers).to(device)
        self.fc = nn.Linear(hidden_size+26, num_classes).to(device)
        
    def forward(self, x, guessed, h0=None, c0=None):
        if h0 == None:
            h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device) 
            c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device) 
        out, (hn,cn) = self.lstm(x, (h0,c0))
        out = out[-1, :, :].to(device)
        out = torch.cat((out,guessed), 1).to(device)
        out = self.fc(out).to(device)
        return out, (hn,cn)


learning_rate = 0.002
hidden_size = 512
num_layers = 2

logsoftmax = nn.LogSoftmax(dim=0)
lstm = LSTM(27,hidden_size,num_layers,26)
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
def criterion(pred, tar):
    return -torch.dot(tar[0],logsoftmax(pred[0]))

lstm.load_state_dict(torch.load(path+f"result_dict_lstm_{ last_epoch }"))


def upd(loss):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

steps = 100
def train(batch_size, epochs):
    all_losses = []
    current_loss = 0
    attempts = 0
    wrong_attempts = 0
    won = 0
    div = 0
    for e in range(last_epoch + 1,epochs):
        random.shuffle(train_words)
        for i in range(200000):
            word = train_words[i]
            guessed = torch.zeros(1,26).to(device)
            obscure_word = '_'*len(word)
            tar = get_freq_tensor(word,obscure_word,guessed)

            if i>0 and i%steps == 0:
                # avg_loss = current_loss/steps
                # upd(avg_loss)
                print(f"epoch: {e}, {i/2000}% completed, avg_loss: {avg_loss}, avg_attempts: {attempts/steps}, avg_wrong_attempts: {wrong_attempts/steps}, won%: {(won*100)/steps}")
                won = 0
                attempts = 0
                wrong_attempts = 0
                all_losses.append(current_loss)
                # current_loss = 0

            if i>0 and i%batch_size == 0:
                avg_loss = current_loss/div
                upd(avg_loss)
                div = 0
                # print(f"epoch: {e}, {i/2000}% completed, avg_loss: {avg_loss}, avg_attempts: {attempts/steps}, avg_wrong_attempts: {wrong_attempts/steps}")
                # attempts = 0
                # wrong_attempts = 0
                # all_losses.append(current_loss)
                current_loss = 0

            x = len(word)
            y = 0
            h0 = 0
            c0 = 0
            cur_a = 0
            while obscure_word.find('_') != -1:
                div+=1
                input = word_to_tensor(obscure_word).to(device)
                attempts += 1
                if type(h0) == int:
                    output, _ = lstm(input, guessed)
                else:
                    output, _ = lstm(input, guessed, h0, c0)

                loss = criterion(output, tar)
                current_loss += loss 
                
                guess = guess_from_output(output, guessed)
                guess_idx = guess_idx_from_output(output, guessed)
                
                # upd(loss)
                guessed[0][ALL_LETTERS.find(guess)] = 1
                if guess in word:
                    x-=1
                    y+=1
                    for j in range(len(word)):
                        if(word[j]==guess):
                            obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                    tar = get_freq_tensor(word,obscure_word,guessed)
                else:
                    wrong_attempts += 1
                    cur_a+=1
                if (i+1)%1000 == 0:
                    print(f"obscure_word: {obscure_word}, guess: {guess}")
            if cur_a < 6:
                won+=1
        torch.save(lstm.state_dict(), path+"result_dict_lstm_"+str(e))
        torch.save(lstm, path+"result_model_lstm_"+str(e))
    plt.figure()
    plt.plot(all_losses)
    plt.show()


train(16, 10)

Streaming output truncated to the last 5000 lines.
obscure_word: d______eeked, guess: l
obscure_word: d_____heeked, guess: h
obscure_word: do____heeked, guess: o
obscure_word: do____heeked, guess: s
obscure_word: do_n__heeked, guess: n
obscure_word: down__heeked, guess: w
obscure_word: downy_heeked, guess: y
obscure_word: downycheeked, guess: c
epoch: 4, 40.0% completed, avg_loss: 2.3495850563049316, avg_attempts: 11.86, avg_wrong_attempts: 4.13, won%: 75.0
epoch: 4, 40.05% completed, avg_loss: 2.2571897506713867, avg_attempts: 11.85, avg_wrong_attempts: 4.13, won%: 72.0
epoch: 4, 40.1% completed, avg_loss: 2.273420572280884, avg_attempts: 11.75, avg_wrong_attempts: 4.29, won%: 69.0
epoch: 4, 40.15% completed, avg_loss: 2.259354591369629, avg_attempts: 12.22, avg_wrong_attempts: 4.79, won%: 66.0
epoch: 4, 40.2% completed, avg_loss: 2.46724271774292, avg_attempts: 12.9, avg_wrong_attempts: 5.29, won%: 68.0
epoch: 4, 40.25% completed, avg_loss: 2.416761875152588, avg_attempts: 12.04, avg

KeyboardInterrupt: ignored

In [4]:
f = open(path+"words.txt", "r")

words = []
words

for _ in range(227300):
    word = f.readline()
    word = word.split('\n')[0]
    words.append(word)

In [5]:
test_words = words[200000:227300]

In [6]:
path = "drive/MyDrive/hangman/testing_hangman/"

In [7]:
hidden_size = 512
num_layers = 2
lstm = LSTM(27,hidden_size,num_layers,26)
lstm.load_state_dict(torch.load(path+f"result_dict_lstm_0"))

<All keys matched successfully>

In [8]:
with torch.no_grad():
    attempts = 0
    wrong_attempts = 0
    won = 0
    for i in range(0,27300):
        word = test_words[i]
        guessed = torch.zeros(1,26).to(device)
        obscure_word = '_'*len(word)
        # print(i)
        if (i+1)%100 == 0:
            print(f"{i+1}::, avg_attempts: {attempts/(i+1)}, avg_wrong_attempts: {wrong_attempts/(i+1)}, won%: {(won*100)/(i+1)}")

        x = len(word)
        y = 0
        h0 = 0
        c0 = 0
        cur_a = 0
        while obscure_word.find('_') != -1:
            input = word_to_tensor(obscure_word).to(device)
            attempts += 1
            
            if type(h0) == int:
                output, _ = lstm(input, guessed)
            else:
                output, _ = lstm(input, guessed, h0, c0)

                # loss = criterion(output, tar)
                # current_loss += loss 
                
            guess = guess_from_output(output, guessed)
            guess_idx = guess_idx_from_output(output, guessed)
                # print(guess)
                # upd(loss)
            guessed[0][ALL_LETTERS.find(guess)] = 1
            if guess in word:
                x-=1
                y+=1
                for j in range(len(word)):
                    if(word[j]==guess):
                        obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                                        # tar = get_freq_tensor(word,obscure_word,guessed)
            else:
                wrong_attempts += 1
                cur_a+=1
            if (i+1)%1000 == 0:
                print(f"obscure_word: {obscure_word}, guess: {guess}")
        if cur_a < 6:
            won+=1

100::, avg_attempts: 13.45, avg_wrong_attempts: 5.77, won%: 52.0
200::, avg_attempts: 13.3, avg_wrong_attempts: 5.82, won%: 53.0
300::, avg_attempts: 13.51, avg_wrong_attempts: 6.073333333333333, won%: 50.666666666666664
400::, avg_attempts: 13.3975, avg_wrong_attempts: 5.9425, won%: 51.0
500::, avg_attempts: 13.34, avg_wrong_attempts: 5.876, won%: 52.4
600::, avg_attempts: 13.378333333333334, avg_wrong_attempts: 5.911666666666667, won%: 51.666666666666664
700::, avg_attempts: 13.391428571428571, avg_wrong_attempts: 5.961428571428572, won%: 51.857142857142854
800::, avg_attempts: 13.41125, avg_wrong_attempts: 5.995, won%: 50.875
900::, avg_attempts: 13.404444444444444, avg_wrong_attempts: 5.993333333333333, won%: 51.111111111111114
1000::, avg_attempts: 13.424, avg_wrong_attempts: 6.003, won%: 50.9
obscure_word: ______e, guess: e
obscure_word: _a____e, guess: a
obscure_word: _a_i__e, guess: i
obscure_word: _a_i_le, guess: l
obscure_word: _a_icle, guess: c
obscure_word: _anicle, guess: 

In [9]:
print(f"epoch 0 results:: avg_attempts: {attempts/(27300)}, avg_wrong_attempts: {wrong_attempts/27300}, won%: {(won*100)/(27300)}")

epoch 0 results:: avg_attempts: 13.393260073260073, avg_wrong_attempts: 5.999084249084249, won%: 52.21978021978022


In [10]:
lstm.load_state_dict(torch.load(path+f"result_dict_lstm_1"))
with torch.no_grad():
    attempts = 0
    wrong_attempts = 0
    won = 0
    for i in range(0,27300):
        word = test_words[i]
        guessed = torch.zeros(1,26).to(device)
        obscure_word = '_'*len(word)
        # print(i)
        if (i+1)%100 == 0:
            print(f"{i+1}::, avg_attempts: {attempts/(i+1)}, avg_wrong_attempts: {wrong_attempts/(i+1)}, won%: {(won*100)/(i+1)}")

        x = len(word)
        y = 0
        h0 = 0
        c0 = 0
        cur_a = 0
        while obscure_word.find('_') != -1:
            input = word_to_tensor(obscure_word).to(device)
            attempts += 1
            
            if type(h0) == int:
                output, _ = lstm(input, guessed)
            else:
                output, _ = lstm(input, guessed, h0, c0)

                # loss = criterion(output, tar)
                # current_loss += loss 
                
            guess = guess_from_output(output, guessed)
            guess_idx = guess_idx_from_output(output, guessed)
                # print(guess)
                # upd(loss)
            guessed[0][ALL_LETTERS.find(guess)] = 1
            if guess in word:
                x-=1
                y+=1
                for j in range(len(word)):
                    if(word[j]==guess):
                        obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                                        # tar = get_freq_tensor(word,obscure_word,guessed)
            else:
                wrong_attempts += 1
                cur_a+=1
            if (i+1)%1000 == 0:
                print(f"obscure_word: {obscure_word}, guess: {guess}")
        if cur_a < 6:
            won+=1

100::, avg_attempts: 12.89, avg_wrong_attempts: 5.21, won%: 56.0
200::, avg_attempts: 12.605, avg_wrong_attempts: 5.125, won%: 58.5
300::, avg_attempts: 12.933333333333334, avg_wrong_attempts: 5.496666666666667, won%: 55.333333333333336
400::, avg_attempts: 12.8325, avg_wrong_attempts: 5.3775, won%: 57.5
500::, avg_attempts: 12.884, avg_wrong_attempts: 5.42, won%: 57.6
600::, avg_attempts: 12.866666666666667, avg_wrong_attempts: 5.4, won%: 57.833333333333336
700::, avg_attempts: 12.877142857142857, avg_wrong_attempts: 5.4471428571428575, won%: 57.285714285714285
800::, avg_attempts: 12.90125, avg_wrong_attempts: 5.485, won%: 56.875
900::, avg_attempts: 12.914444444444445, avg_wrong_attempts: 5.503333333333333, won%: 56.333333333333336
1000::, avg_attempts: 12.936, avg_wrong_attempts: 5.515, won%: 56.6
obscure_word: ______e, guess: e
obscure_word: ___i__e, guess: i
obscure_word: _a_i__e, guess: a
obscure_word: _a_i_le, guess: l
obscure_word: _a_i_le, guess: b
obscure_word: _a_icle, gues

In [11]:
print(f"epoch 1 results:: avg_attempts: {attempts/(27300)}, avg_wrong_attempts: {wrong_attempts/27300}, won%: {(won*100)/(27300)}")

epoch 1 results:: avg_attempts: 12.946263736263736, avg_wrong_attempts: 5.552087912087912, won%: 57.51648351648352


In [12]:
lstm.load_state_dict(torch.load(path+f"result_dict_lstm_2"))
with torch.no_grad():
    attempts = 0
    wrong_attempts = 0
    won = 0
    for i in range(0,27300):
        word = test_words[i]
        guessed = torch.zeros(1,26).to(device)
        obscure_word = '_'*len(word)
        # print(i)
        if (i+1)%100 == 0:
            print(f"{i+1}::, avg_attempts: {attempts/(i+1)}, avg_wrong_attempts: {wrong_attempts/(i+1)}, won%: {(won*100)/(i+1)}")

        x = len(word)
        y = 0
        h0 = 0
        c0 = 0
        cur_a = 0
        while obscure_word.find('_') != -1:
            input = word_to_tensor(obscure_word).to(device)
            attempts += 1
            
            if type(h0) == int:
                output, _ = lstm(input, guessed)
            else:
                output, _ = lstm(input, guessed, h0, c0)

                # loss = criterion(output, tar)
                # current_loss += loss 
                
            guess = guess_from_output(output, guessed)
            guess_idx = guess_idx_from_output(output, guessed)
                # print(guess)
                # upd(loss)
            guessed[0][ALL_LETTERS.find(guess)] = 1
            if guess in word:
                x-=1
                y+=1
                for j in range(len(word)):
                    if(word[j]==guess):
                        obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                                        # tar = get_freq_tensor(word,obscure_word,guessed)
            else:
                wrong_attempts += 1
                cur_a+=1
            if (i+1)%1000 == 0:
                print(f"obscure_word: {obscure_word}, guess: {guess}")
        if cur_a < 6:
            won+=1

100::, avg_attempts: 12.99, avg_wrong_attempts: 5.31, won%: 55.0
200::, avg_attempts: 12.825, avg_wrong_attempts: 5.345, won%: 57.0
300::, avg_attempts: 12.913333333333334, avg_wrong_attempts: 5.476666666666667, won%: 54.666666666666664
400::, avg_attempts: 12.86, avg_wrong_attempts: 5.405, won%: 55.5
500::, avg_attempts: 12.83, avg_wrong_attempts: 5.366, won%: 57.2
600::, avg_attempts: 12.835, avg_wrong_attempts: 5.368333333333333, won%: 57.666666666666664
700::, avg_attempts: 12.805714285714286, avg_wrong_attempts: 5.3757142857142854, won%: 57.57142857142857
800::, avg_attempts: 12.79625, avg_wrong_attempts: 5.38, won%: 57.5
900::, avg_attempts: 12.784444444444444, avg_wrong_attempts: 5.373333333333333, won%: 57.44444444444444
1000::, avg_attempts: 12.814, avg_wrong_attempts: 5.393, won%: 57.5
obscure_word: ______e, guess: e
obscure_word: _a____e, guess: a
obscure_word: _a_i__e, guess: i
obscure_word: _a_i_le, guess: l
obscure_word: _a_i_le, guess: b
obscure_word: _a_icle, guess: c
o

In [13]:
print(f"epoch 2 results:: avg_attempts: {attempts/(27300)}, avg_wrong_attempts: {wrong_attempts/27300}, won%: {(won*100)/(27300)}")

epoch 2 results:: avg_attempts: 12.717509157509157, avg_wrong_attempts: 5.323333333333333, won%: 60.476190476190474


In [14]:
lstm.load_state_dict(torch.load(path+f"result_dict_lstm_3"))
with torch.no_grad():
    attempts = 0
    wrong_attempts = 0
    won = 0
    for i in range(0,27300):
        word = test_words[i]
        guessed = torch.zeros(1,26).to(device)
        obscure_word = '_'*len(word)
        # print(i)
        if (i+1)%100 == 0:
            print(f"{i+1}::, avg_attempts: {attempts/(i+1)}, avg_wrong_attempts: {wrong_attempts/(i+1)}, won%: {(won*100)/(i+1)}")

        x = len(word)
        y = 0
        h0 = 0
        c0 = 0
        cur_a = 0
        while obscure_word.find('_') != -1:
            input = word_to_tensor(obscure_word).to(device)
            attempts += 1
            
            if type(h0) == int:
                output, _ = lstm(input, guessed)
            else:
                output, _ = lstm(input, guessed, h0, c0)

                # loss = criterion(output, tar)
                # current_loss += loss 
                
            guess = guess_from_output(output, guessed)
            guess_idx = guess_idx_from_output(output, guessed)
                # print(guess)
                # upd(loss)
            guessed[0][ALL_LETTERS.find(guess)] = 1
            if guess in word:
                x-=1
                y+=1
                for j in range(len(word)):
                    if(word[j]==guess):
                        obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                                        # tar = get_freq_tensor(word,obscure_word,guessed)
            else:
                wrong_attempts += 1
                cur_a+=1
            if (i+1)%1000 == 0:
                print(f"obscure_word: {obscure_word}, guess: {guess}")
        if cur_a < 6:
            won+=1

100::, avg_attempts: 12.84, avg_wrong_attempts: 5.16, won%: 57.0
200::, avg_attempts: 12.545, avg_wrong_attempts: 5.065, won%: 60.0
300::, avg_attempts: 12.7, avg_wrong_attempts: 5.263333333333334, won%: 57.666666666666664
400::, avg_attempts: 12.58, avg_wrong_attempts: 5.125, won%: 58.75
500::, avg_attempts: 12.574, avg_wrong_attempts: 5.11, won%: 59.8
600::, avg_attempts: 12.56, avg_wrong_attempts: 5.093333333333334, won%: 60.833333333333336
700::, avg_attempts: 12.564285714285715, avg_wrong_attempts: 5.134285714285714, won%: 60.285714285714285
800::, avg_attempts: 12.565, avg_wrong_attempts: 5.14875, won%: 60.75
900::, avg_attempts: 12.593333333333334, avg_wrong_attempts: 5.182222222222222, won%: 59.888888888888886
1000::, avg_attempts: 12.624, avg_wrong_attempts: 5.203, won%: 59.7
obscure_word: ______e, guess: e
obscure_word: ___i__e, guess: i
obscure_word: _a_i__e, guess: a
obscure_word: _a_i_le, guess: l
obscure_word: _a_i_le, guess: b
obscure_word: pa_i_le, guess: p
obscure_word

In [15]:
print(f"epoch 3 results:: avg_attempts: {attempts/(27300)}, avg_wrong_attempts: {wrong_attempts/27300}, won%: {(won*100)/(27300)}")

epoch 3 results:: avg_attempts: 12.612857142857143, avg_wrong_attempts: 5.218681318681319, won%: 61.49084249084249


In [16]:
lstm.load_state_dict(torch.load(path+f"result_dict_lstm_4"))
with torch.no_grad():
    attempts = 0
    wrong_attempts = 0
    won = 0
    for i in range(0,27300):
        word = test_words[i]
        guessed = torch.zeros(1,26).to(device)
        obscure_word = '_'*len(word)
        # print(i)
        if (i+1)%100 == 0:
            print(f"{i+1}::, avg_attempts: {attempts/(i+1)}, avg_wrong_attempts: {wrong_attempts/(i+1)}, won%: {(won*100)/(i+1)}")

        x = len(word)
        y = 0
        h0 = 0
        c0 = 0
        cur_a = 0
        while obscure_word.find('_') != -1:
            input = word_to_tensor(obscure_word).to(device)
            attempts += 1
            
            if type(h0) == int:
                output, _ = lstm(input, guessed)
            else:
                output, _ = lstm(input, guessed, h0, c0)

                # loss = criterion(output, tar)
                # current_loss += loss 
                
            guess = guess_from_output(output, guessed)
            guess_idx = guess_idx_from_output(output, guessed)
                # print(guess)
                # upd(loss)
            guessed[0][ALL_LETTERS.find(guess)] = 1
            if guess in word:
                x-=1
                y+=1
                for j in range(len(word)):
                    if(word[j]==guess):
                        obscure_word = obscure_word[:j] + guess + obscure_word[j+1:]
                                        # tar = get_freq_tensor(word,obscure_word,guessed)
            else:
                wrong_attempts += 1
                cur_a+=1
            if (i+1)%1000 == 0:
                print(f"obscure_word: {obscure_word}, guess: {guess}")
        if cur_a < 6:
            won+=1

100::, avg_attempts: 12.88, avg_wrong_attempts: 5.2, won%: 62.0
200::, avg_attempts: 12.7, avg_wrong_attempts: 5.22, won%: 63.5
300::, avg_attempts: 12.76, avg_wrong_attempts: 5.323333333333333, won%: 61.666666666666664
400::, avg_attempts: 12.705, avg_wrong_attempts: 5.25, won%: 63.0
500::, avg_attempts: 12.662, avg_wrong_attempts: 5.198, won%: 63.4
600::, avg_attempts: 12.613333333333333, avg_wrong_attempts: 5.1466666666666665, won%: 62.666666666666664
700::, avg_attempts: 12.634285714285713, avg_wrong_attempts: 5.204285714285715, won%: 62.142857142857146
800::, avg_attempts: 12.6675, avg_wrong_attempts: 5.25125, won%: 62.0
900::, avg_attempts: 12.673333333333334, avg_wrong_attempts: 5.262222222222222, won%: 61.55555555555556
1000::, avg_attempts: 12.711, avg_wrong_attempts: 5.29, won%: 61.1
obscure_word: ______e, guess: e
obscure_word: _a____e, guess: a
obscure_word: _a_i__e, guess: i
obscure_word: _a_i_le, guess: l
obscure_word: _a_i_le, guess: b
obscure_word: _a_icle, guess: c
obs

In [17]:
print(f"epoch 4 results:: avg_attempts: {attempts/(27300)}, avg_wrong_attempts: {wrong_attempts/27300}, won%: {(won*100)/(27300)}")

epoch 4 results:: avg_attempts: 12.562527472527472, avg_wrong_attempts: 5.168351648351648, won%: 62.688644688644686
